<a href="https://colab.research.google.com/github/yvk9/learn/blob/master/MagicSquare3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install z3-solver


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.3/29.3 MB 38.4 MB/s eta 0:00:00


In [3]:
from z3 import Solver, Int, Distinct, Sum, sat


def synthesize_magic_square(size: int) -> None:
    """
    Synthesize an n×n (size×size) normal magic square using Z3 constraints.

    Properties:
    - Uses each number from 1..n^2 exactly once (all distinct, bounded domain)
    - Every row, column, and both main diagonals sum to the same value
    """

    # Normal magic squares (using 1..n^2) do not exist for n = 2
    if size == 2:
        print("No normal 2×2 magic square exists.")
        return

    solver = Solver()

    # Create the grid of integer variables
    square = [
        [Int(f"cell_{r}_{c}") for c in range(size)]
        for r in range(size)
    ]

    # Flatten the grid for domain + distinct constraints
    all_cells = [square[r][c] for r in range(size) for c in range(size)]

    # Domain constraints: each value must be between 1 and n^2
    solver.add([cell >= 1 for cell in all_cells])
    solver.add([cell <= size * size for cell in all_cells])

    # All values must be distinct
    solver.add(Distinct(all_cells))

    # Magic sum for a normal magic square using 1..n^2
    magic_sum = size * (size * size + 1) // 2

    # Row sum constraints
    for r in range(size):
        solver.add(Sum(square[r]) == magic_sum)

    # Column sum constraints
    for c in range(size):
        solver.add(Sum(square[r][c] for r in range(size)) == magic_sum)

    # Diagonal sum constraints
    solver.add(Sum(square[i][i] for i in range(size)) == magic_sum)
    solver.add(Sum(square[i][size - 1 - i] for i in range(size)) == magic_sum)

    # Solve and print the square
    if solver.check() == sat:
        model = solver.model()
        for r in range(size):
            print(*[model.evaluate(square[r][c]).as_long() for c in range(size)])
    else:
        print("UNSAT (no magic square found).")


synthesize_magic_square(3)

4 9 2
3 5 7
8 1 6
